In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
sns.set(color_codes=True)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import random



from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 

In [5]:
df = pd.read_csv('restaurant-menus.csv')


In [87]:
restaurant_id = pd.read_csv('restaurants.csv')

In [9]:
df

,restaurant_id,category,name,description,price
0,1,Extra Large Pizza,Extra Large Meat Lovers,Whole pie.,15.99 USD
1,1,Extra Large Pizza,Extra Large Supreme,Whole pie.,15.99 USD
2,1,Extra Large Pizza,Extra Large Pepperoni,Whole pie.,14.99 USD
3,1,Extra Large Pizza,Extra Large BBQ Chicken &amp; Bacon,Whole Pie,15.99 USD
4,1,Extra Large Pizza,Extra Large 5 Cheese,Whole pie.,14.99 USD
...,...,...,...,...,...
5117212,63469,Other Essentials,Composition Notebook,NaN,4.38 USD
5117213,63469,Other Essentials,Fancy Fest Savory Salmon - 3oz,NaN,1.19 USD
5117214,63469,Other Essentials,Bicycle Playing Cards,NaN,3.83 USD
5117215,63469,Other Essentials,Tidy Cat Liter - 10lbs,NaN,4.38 USD


In [10]:
df.dtypes

restaurant_id     int64
category         object
name             object
description      object
price            object
dtype: object

In [11]:
df['name'] = df['name'].astype(str)

In [12]:
df.dtypes

restaurant_id     int64
category         object
name             object
description      object
price            object
dtype: object

In [13]:
df.count()

restaurant_id    5117217
category         5117217
name             5117217
description      3665077
price            5117217
dtype: int64

In [14]:
df = df.dropna()
df = df.reset_index(drop=True)
df.count()

restaurant_id    3665077
category         3665077
name             3665077
description      3665077
price            3665077
dtype: int64

In [360]:
ra = random.randint(0, 5117210)
print(df['name'][ra], " " , df['description'][ra])

Hibachi Filet Mignon   Cooked with Japanese soy sauce and teriyaki sauce. Served with onion mushroom soup and green salad. Include mixed hibachi vegetables and 2 pieces of shrimp. Steam rice or fried rice.


In [60]:
input_str =  'bacon egg ranch cheese chicken sandwich'

best_match = ''
best_score = 0

for i in range(1, 100000):
    sc = fuzz.token_set_ratio( input_str, df['description'][i])
    
    if sc > best_score:                    
        best_score = sc
        best_match = df['name'][i] + ': ' + df['description'][i]

print(best_match)
print(best_score)


Chicken Bacon Ranch Cheesesteak: Chicken sandwich with bacon, ranch, and your choice of cheese.
95


In [68]:
input_str =  'bacon egg cheese'

best_match = ''
best_score = 0

for i in range(0, 1000):
    sc = fuzz.token_set_ratio( input_str, df['description'][i])
    if sc > best_score:                    
        best_score = sc
        best_match = df['name'][i] + ': ' + df['description'][i]

print(best_match)
print(best_score)

Stuffed Waffle Biscuits: Your Choose your size of meats are sausage or ham or bacon or eggs, turkey (two meats with cheese).
86


In [69]:
input_str =  'chicken and rice'

best_match = ''
best_score = 0

for i in range(0, 1000):
    sc = fuzz.token_set_ratio( input_str, df['description'][i])
    if sc > best_score:                    
        best_score = sc
        best_match = df['name'][i] + ': ' + df['description'][i]

print(best_match)
print(best_score)

Cream of Chicken &amp; Wild Rice Soup: Bowl (260 Cal.), Cup (180 Cal.), Bread Bowl (840 Cal.), Group (620 Cal.) Diced chicken, long grain and wild rice, celery, onion and carrots simmered in chicken stock with cream and select herbs. Allergens: Contains Wheat, Milk
100


In [71]:
# pip install wikipedia-api

In [70]:
# import wikipediaapi
# import pandas as pd

# def fetch_wikipedia_data(titles):
#     wiki_wiki = wikipediaapi.Wikipedia('en')
#     data = []
#     for title in titles:
#         page = wiki_wiki.page(title)
#         if page.exists():
#             data.append([title, page.fullurl, page.summary])
#     return data

# # Example list of Wikipedia article titles (you may need to generate or obtain this list)
# article_titles = [
#     "Artificial intelligence",
#     "Machine learning",
#     "Data science",
#     # Add more titles as needed
# ]

# # Fetch data for the specified article titles
# data = fetch_wikipedia_data(article_titles)

# # Convert the data into a Pandas DataFrame
# df = pd.DataFrame(data, columns=['Title', 'Link', 'Summary'])

# # Print the DataFrame
# print(df)


In [82]:
df['menu_item'] = df['name'] + ' ' + df['description']
df


,restaurant_id,category,name,description,price,menu_item
0,1,Extra Large Pizza,Extra Large Meat Lovers,Whole pie.,15.99 USD,Extra Large Meat Lovers Whole pie.
1,1,Extra Large Pizza,Extra Large Supreme,Whole pie.,15.99 USD,Extra Large Supreme Whole pie.
2,1,Extra Large Pizza,Extra Large Pepperoni,Whole pie.,14.99 USD,Extra Large Pepperoni Whole pie.
3,1,Extra Large Pizza,Extra Large BBQ Chicken &amp; Bacon,Whole Pie,15.99 USD,Extra Large BBQ Chicken &amp; Bacon Whole Pie
4,1,Extra Large Pizza,Extra Large 5 Cheese,Whole pie.,14.99 USD,Extra Large 5 Cheese Whole pie.
...,...,...,...,...,...,...
3665072,63468,Desserts &amp; More,Apple Pies,Satisfy your sweet tooth with our apple pie. J...,1.69 USD,Apple Pies Satisfy your sweet tooth with our a...
3665073,63469,Food,Mini Tacos - 10 pc,10 Count,5.32 USD,Mini Tacos - 10 pc 10 Count
3665074,63469,Food,Tyson Chicken Wings,Served with your choice of flavor.,4.71 USD,Tyson Chicken Wings Served with your choice of...
3665075,63469,Food,Mini Tacos - 10 pc,10 Count,5.32 USD,Mini Tacos - 10 pc 10 Count


In [98]:
# Group by 'restaurant_id' and concatenate 'menu_item' into one string separated by ','
df_grouped = df.groupby('restaurant_id')['menu_item'].agg(lambda x: ' '.join(x)).reset_index()

# Rename the aggregated column
df_grouped.rename(columns={'menu_item': 'menu'}, inplace=True)

print(df_grouped)


       restaurant_id                                               menu
0                  1  Extra Large Meat Lovers Whole pie. Extra Large...
1                  2  J’ Ti`’z Smoothie Tropical fruit blend, dragon...
2                  3  Chicken Bacon Ranch Cheesesteak Chicken sandwi...
3                  4  Chicken Bacon Artichoke - Keto Friendly (Bakin...
4                  5  Full Sausage (2 pcs) with 2 Eggs 2 pieces. Dou...
...              ...                                                ...
60922          63465  Breakfast Sampler Two eggs* your way, 2 custom...
60923          63466  Ready Rabbit Blue Curacao Non Alcoholic 750ml ...
60924          63467  Heineken Non-Alcoholic 0.0 6x 11.2oz Cans Our ...
60925          63468  3 Piece Combo Three pieces of chicken with two...
60926          63469  Mini Tacos - 10 pc 10 Count Tyson Chicken Wing...

[60927 rows x 2 columns]


In [106]:
df_grouped['menu'][26]

'Build Your Own Burrito Two scrambled eggs with your choice of meat and toppings wrapped in a fresh tortilla. Basic Breakfast Burrito Two scrambled eggs, breakfast potatoes, and melted cheese wrapped in a fresh flour tortilla. Bacon Breakfast Burrito Two scrambled eggs, crispy bacon, breakfast potatoes, and melted cheese wrapped in a fresh flour tortilla. Carne Asada Breakfast Burrito Two scrambled eggs, carne asada, breakfast potatoes, and melted cheese wrapped in a fresh flour tortilla. Sausage Breakfast Burrito Two scrambled eggs, sausage, breakfast potatoes, and melted cheese wrapped in a fresh flour tortilla. Spicy Hot Breakfast Burrito Two scrambled eggs, jalapenos, breakfast potatoes, sriracha and melted cheese wrapped in a fresh flour tortilla. Everything but the Kitchen Sink Breakfast Burrito Two scrambled eggs, bacon, carne asada, breakfast potatoes, avocado and melted cheese wrapped in a flour tortilla. Eat Your Veggies Breakfast Burrito Two scrambled eggs, sauteed mushrooms

In [105]:
input_str =  'carne asada burrito'

best_match_rest = ''
best_match_menu = ''
best_score = 0

for i in range(0, 10000):
    sc = fuzz.token_set_ratio( input_str, df_grouped['menu'][i])
    if sc > best_score:                    
        best_score = sc
        best_match_rest = str(i) + ": " + restaurant_id['name'][i]
        best_match_menu = df_grouped['menu'][i]

print(best_match_rest)
print(best_match_menu)
print(best_score)

26: Ruscelli's Food Truck at Mojo Pub
Build Your Own Burrito Two scrambled eggs with your choice of meat and toppings wrapped in a fresh tortilla. Basic Breakfast Burrito Two scrambled eggs, breakfast potatoes, and melted cheese wrapped in a fresh flour tortilla. Bacon Breakfast Burrito Two scrambled eggs, crispy bacon, breakfast potatoes, and melted cheese wrapped in a fresh flour tortilla. Carne Asada Breakfast Burrito Two scrambled eggs, carne asada, breakfast potatoes, and melted cheese wrapped in a fresh flour tortilla. Sausage Breakfast Burrito Two scrambled eggs, sausage, breakfast potatoes, and melted cheese wrapped in a fresh flour tortilla. Spicy Hot Breakfast Burrito Two scrambled eggs, jalapenos, breakfast potatoes, sriracha and melted cheese wrapped in a fresh flour tortilla. Everything but the Kitchen Sink Breakfast Burrito Two scrambled eggs, bacon, carne asada, breakfast potatoes, avocado and melted cheese wrapped in a flour tortilla. Eat Your Veggies Breakfast Burrito 

In [102]:
input_str =  'Carne Asada Breakfast Burrito'

best_match_rest = ''
best_match_menu = ''
best_score = 0

for i in range(0, 10000):
    sc = fuzz.token_set_ratio( input_str, df_grouped['menu'][i])
    if sc > best_score:                    
        best_score = sc
        best_match_rest = str(i) + ": " + restaurant_id['name'][i]
        best_match_menu = df_grouped['menu'][i]

print(best_match_rest)
print(best_match_menu)
print(best_score)

26: Ruscelli's Food Truck at Mojo Pub
Build Your Own Burrito Two scrambled eggs with your choice of meat and toppings wrapped in a fresh tortilla. Basic Breakfast Burrito Two scrambled eggs, breakfast potatoes, and melted cheese wrapped in a fresh flour tortilla. Bacon Breakfast Burrito Two scrambled eggs, crispy bacon, breakfast potatoes, and melted cheese wrapped in a fresh flour tortilla. Carne Asada Breakfast Burrito Two scrambled eggs, carne asada, breakfast potatoes, and melted cheese wrapped in a fresh flour tortilla. Sausage Breakfast Burrito Two scrambled eggs, sausage, breakfast potatoes, and melted cheese wrapped in a fresh flour tortilla. Spicy Hot Breakfast Burrito Two scrambled eggs, jalapenos, breakfast potatoes, sriracha and melted cheese wrapped in a fresh flour tortilla. Everything but the Kitchen Sink Breakfast Burrito Two scrambled eggs, bacon, carne asada, breakfast potatoes, avocado and melted cheese wrapped in a flour tortilla. Eat Your Veggies Breakfast Burrito 

In [107]:
df_grouped['menu'] = [entry.lower() for entry in df_grouped['menu']]
df_grouped['menu'] = [word_tokenize(entry) for entry in df_grouped['menu']]

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(df_grouped['menu']):
    if index % 500 == 0:
      print(index)
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df_grouped.loc[index,'menu_final'] = str(Final_words)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500


In [108]:
import pickle
from nltk.stem import WordNetLemmatizer


# Save the lemmatizer to a file
with open('wordnet_lemmatizer.pkl', 'wb') as f:
    pickle.dump(word_Lemmatized, f)


In [109]:
df_grouped.to_csv('data_cl.csv', index=False)